In [1]:
import pandas as pd
from pathlib import Path
import os

directory = Path(os.getcwd())   
directory = directory.parent.parent.joinpath("data")  
df = pd.read_pickle(directory.joinpath("stocktwits_processed_without_multiple_full.pkl"))
df

,id,text,time,sentiment,symbols
0,411896118,I’m comfortable at $127.50. I’ll check back in...,1638291431,Bullish,$BABA
1,411895020,how low does this POS go tomorrow is the milli...,1638291222,,$BABA
2,411894156,@Nobrainer6868 started my first 5% position t...,1638291057,,$BABA
3,411894124,You guys want me to sell down here? Look at th...,1638291050,,$BABA
4,411893846,at this price point shorts are better off flip...,1638291000,,$BABA
...,...,...,...,...,...
386926,420618061,USBancorp BidaskScore is #Reiterated to Stron...,1640479505,,$USB
386927,420617596,Short sale volume (not short interest) for on...,1640478214,,$USB
386928,420610698,SweepCast observed: with Unusual Options Acti...,1640466364,,$USB
386929,420437859,How does this affect your portfolio? 's in Upt...,1640326224,,$USB


In [2]:
df.text.value_counts().index

Index(['',
       'Form FWP (filing under securities act rules 163/433 of free writing prospectuses) filed with the SEC',
       'Form 424B2 (prospectus [rule 424(b)(2)]) filed with the SEC',
       '📜 SEC Form FWP filed by Morgan Stanley    45 seconds delayed.', 'lol',
       '.',
       'made it on to the Best Stocks short list with an Earning Momentum of 22 and a perfect timing.',
       'wow', 'Simulated 162.5 dollar CALLS for Monday's open on StockOrbit.',
       'Form 6-K (report of foreign issuer [rules 13a-16 and 15d-16]) filed with the SEC',
       ...
       'finishing at the LOD???? Frick!', 'split news coming!!!!',
       'Release the split rumor on Benzinga!', 'WTFFFF ARE U DOING',
       '3425 close. EASY', 'it’s not popping but not dropping 🧐',
       'honestly would be a perfect time to announce the split news here, bringing the price to $4000 before eoy',
       'about to see 3420 at close.   3440 after hours.',
       'after hours split news today??????',
       '56 m

In [3]:
# LABELS
VALID = 1
INVALID = 0
UNLABEL = -1

In [4]:
text_file = open("tfidf_words/tfidf_words.txt", "r")
tfidf_words = text_file.readlines()
tfidf_words = [i.replace('\n', '') for i in tfidf_words]
tfidf_words[:10]

['oracle',
 'cerner',
 'orcl',
 '100',
 'cloud',
 'coca',
 'cola',
 'activity',
 'worth',
 'bodyarmor']

In [5]:
from snorkel.labeling import labeling_function
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier
import re

@labeling_function()
def contains_num(x):
    return VALID if bool(re.search(r'\d', x.text)) else UNLABEL


@labeling_function()
def short_sentence(x):
    return INVALID if (len(x.text.split()) < 5) and not ((len(x.text.split()) == 3 or len(x.text.split()) == 4) and bool(re.search(r'\d', x.text))) else UNLABEL

@labeling_function()
def blank(x):
    return INVALID if x.text == '' else UNLABEL

@labeling_function()
def in_tfidf_words(x):
    return VALID if set(x.text.lower().split()) & set(tfidf_words) else UNLABEL



# Define the set of labeling functions (LFs)
lfs = [contains_num, short_sentence, blank, in_tfidf_words]

# Apply the LFs to the unlabeled training data
applier = PandasLFApplier(lfs)
L_train = applier.apply(df)

from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 384876/384876 [00:24<00:00, 15810.67it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
contains_num,0,[1],0.401056,0.201041,0.018479
short_sentence,1,[0],0.236629,0.076994,0.036477
blank,2,[0],0.040517,0.040517,0.000000
in_tfidf_words,3,[1],0.348268,0.202751,0.020188


In [6]:
# Train the label model and compute the training labels
label_model = LabelModel()
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
df["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")


INFO:root:Computing O...
INFO:root:Estimating \mu...
  8%|▊         | 41/500 [00:00<00:01, 405.95epoch/s]INFO:root:[50 epochs]: TRAIN:[loss=0.003]
INFO:root:[100 epochs]: TRAIN:[loss=0.001]
INFO:root:[150 epochs]: TRAIN:[loss=0.000]
INFO:root:[200 epochs]: TRAIN:[loss=0.000]
 40%|████      | 202/500 [00:00<00:00, 1111.29epoch/s]INFO:root:[250 epochs]: TRAIN:[loss=0.000]
INFO:root:[300 epochs]: TRAIN:[loss=0.000]
INFO:root:[350 epochs]: TRAIN:[loss=0.000]
 77%|███████▋  | 383/500 [00:00<00:00, 1429.42epoch/s]INFO:root:[400 epochs]: TRAIN:[loss=0.000]
INFO:root:[450 epochs]: TRAIN:[loss=0.000]
100%|██████████| 500/500 [00:00<00:00, 1369.75epoch/s]
INFO:root:Finished Training


Check wether the results are correct

In [8]:
list(df.iloc[L_train[:,0] == VALID].sample(20).text)

['@ AMD NXPI QCOM Daily & 30 m Charts . Are we living  in a simulation or what ...',
 'short term target at $135. Shorts will pay...👍👍👍👍💰💰💰 multi day runner',
 '📜 SEC Form 4: Page Lawrence converted options into 13,889 shares and sold $41,121,590 worth of shares (13,889 units at $2,960.73) as part of a pre-agreed trading plan    45 seconds delayed.',
 'MSFT Trending on Twitter based on the percentage increase in impressions over the last 24hrs.  The dashboard tracks trending stock on Twitter, StockTwits, and Reddit. It breaks it down in real-time so you can catch the trend before it takes off. Check it out, LMK:',
 '50k 🔮',
 'loaded up on 12/3 170 P. 165 by Friday',
 'still an uptrend from the 118, slowly we will get there',
 '108s were nice ill add more over time.',
 '3000 by Friday',
 '39 incoming',
 'NEW ARTICLE : Options Market Suggests Favorable Outlook On QQQ To Mid-2022',
 'atleast 1100 open Monday',
 'Oh how many rookies are here …..  Move your chart out and you will see this i

Still unlabelled after running through all labelling function

In [9]:
list(df.iloc[[not any(i) for i in L_train[:] != UNLABEL]].sample(2).text)

['not good. Looking like a red close',
 'Futures up. Check major support  and resistance levels in nasdaq  before market open']

data that is labelled as valid

In [27]:
list(df[df['label'] == 1].sample(10).text)

['📜 SEC Form FWP filed by Morgan Stanley    45 seconds delayed.',
 'the algorithms at area 51 are designed to break your conviction in your investment',
 'sry for bears who hopes today go to 800 $ …. Fck no sorry 😂😂😂',
 'split announcement incoming 🔥 12/09 is tomorrow!',
 '$600 1/2 years.',
 'New Press Release:      The Walt Disney Company Names Horacio Gutierrez as General Counsel and Secretary     Horacio Gutierrez has been named Senior Executive Vice President, General Counsel and Secretary of The Walt Disney Company (NYSE: DIS), it was announced today by Bob Chapek, Chief Executive Officer. Mr. Gutierrez joins Disney from Spotify, where he has served as Head of Global Affairs and Chief Legal Officer. His appointment is effective February 1, 2022, and he will report directly to Mr. Chapek. Mr. Gutierrez will succeed longtime Disney veteran Alan Braverman who, as was announced in July, is retiring after nearly two decades as General Counsel. This press release features multimedia. Vi

data that is lablled as invalid

In [15]:
list(df[df['label'] == 0].sample(20).text)

['wtf!!! $mnmd $nvax fml!!',
 '💪',
 'can’t fool me twice',
 'help el salvador',
 'ouch',
 '😳',
 'they got your shares!!!',
 ',',
 'FUCK YOU BULLLSSASASSSSAASSA',
 'You did this😡🤣🤣🤣',
 'must be lunch time',
 'relaxed here',
 'reminder...stay patient...',
 'rabble rabble rabble',
 'adding calls here',
 '',
 'fake dump',
 'stop selling, bitches',
 'Buy more',
 '💥']

data that is unlabelled

In [12]:
list(df[df['label'] == -1].sample(20).text)

['fade the open. That jump is silly. Just a fade. It is so obvious the markets are manipulated. It is like they forgot about the Fed and inflation in a day.',
 'out of all the disappointing days for the past few months, this one is the most demoralizing. Congrats bears.',
 'why buy the dips now when you can wait for a bigger dip.',
 'At least go green for the day to shut some bears up',
 'currently assembles the majority of iPhones in its Shenzen, China, location, although Foxconn maintains factories in countries across the world, including Thailand, Malaysia, the Czech Republic, South Korea, Singapore, and the Philippines.  Why not setup some assembly in the USA?',
 'looks like the pendulum had swing back in our favor.',
 'hanging on to dear life',
 'people gonna get Filleted here be careful',
 'don’t do this to me you slut go back up',
 'I personally have not thought of gold as being a store of value for years now.',
 'weird that exactly the same percentage of drop in SPY, DJI and Na

In [13]:
df[df['label'] == 1].to_pickle(directory.joinpath("stocktwits_processed_cleaned_full.pkl"))